In [1]:
import gym
import random
env = gym.make('Hopper-v1')
obs = env.reset()
print (obs)

[2017-03-22 10:33:09,960] Making new env: Hopper-v1


[  1.24874924e+00   1.97693951e-03  -3.58382977e-03   1.22285822e-03
   4.73324722e-03   3.89898475e-03   2.39304647e-03   3.38352279e-04
  -2.88478398e-03  -3.12353093e-03   2.83399406e-03]


In [2]:
for _ in range(1000):
    env.render()
    env.step(env.action_space.sample())

In [4]:
print (env.action_space)

Box(3,)


In [5]:
print (env.action_space.high)

[ 1.  1.  1.]


print (env.action_space.low)

## Generate expert data

In [7]:
import pickle
import tensorflow as tf
import numpy as np
import tf_util
import gym
import load_policy
from dotmap import DotMap

def get_expert_data(expert_policy_file, envname, render=True, max_timesteps=None, num_rollouts=20):
    """
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument('expert_policy_file', type=str)
    parser.add_argument('envname', type=str)
    parser.add_argument('--render', action='store_true')
    parser.add_argument("--max_timesteps", type=int)
    parser.add_argument('--num_rollouts', type=int, default=20,
                        help='Number of expert roll outs')
    args = parser.parse_args()
    """
    args = {'expert_policy_file': expert_policy_file, 
           'envname': envname, 'render': render, 
           'max_timesteps': max_timesteps, 
           'num_rollouts': num_rollouts}
    args = DotMap(args)
    print('loading and building expert policy')
    
    tf.reset_default_graph()
    policy_fn = load_policy.load_policy(args.expert_policy_file)
    print('loaded and built')

    with tf.Session():
        tf_util.initialize()

        import gym
        env = gym.make(args.envname)
        max_steps = args.max_timesteps or env.spec.timestep_limit

        returns = []
        observations = []
        actions = []
        for i in range(args.num_rollouts):
            print('iter', i)
            obs = env.reset()
            done = False
            totalr = 0.
            steps = 0
            while not done:
                action = policy_fn(obs[None,:])
                observations.append(obs)
                actions.append(action)
                obs, r, done, _ = env.step(action)
                totalr += r
                steps += 1
                if args.render:
                    env.render()
                # if steps % 100 == 0: print("%i/%i"%(steps, max_steps))
                if steps >= max_steps:
                    break
            returns.append(totalr)

        print('returns', returns)
        print('mean return', np.mean(returns))
        print('std of return', np.std(returns))

        expert_data = {'observations': np.array(observations),
                       'actions': np.array(actions)}
        return expert_data

In [7]:
expert_data = get_expert_data('experts/Hopper-v1.pkl', 'Hopper-v1', 
                             render=False, num_rollouts=400)

loading and building expert policy
obs (1, 11) (1, 11)
loaded and built
Instructions for updating:
Please use tf.global_variables instead.


[2017-03-21 15:20:35,740] From /home/luo/dlnd/homework/hw1/tf_util.py:91: all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Please use tf.global_variables instead.


Instructions for updating:
Use `tf.variables_initializer` instead.


[2017-03-21 15:20:35,742] From /home/luo/dlnd/homework/hw1/tf_util.py:92: initialize_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.variables_initializer` instead.
[2017-03-21 15:20:35,767] Making new env: Hopper-v1


iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9
iter 10
iter 11
iter 12
iter 13
iter 14
iter 15
iter 16
iter 17
iter 18
iter 19
iter 20
iter 21
iter 22
iter 23
iter 24
iter 25
iter 26
iter 27
iter 28
iter 29
iter 30
iter 31
iter 32
iter 33
iter 34
iter 35
iter 36
iter 37
iter 38
iter 39
iter 40
iter 41
iter 42
iter 43
iter 44
iter 45
iter 46
iter 47
iter 48
iter 49
iter 50
iter 51
iter 52
iter 53
iter 54
iter 55
iter 56
iter 57
iter 58
iter 59
iter 60
iter 61
iter 62
iter 63
iter 64
iter 65
iter 66
iter 67
iter 68
iter 69
iter 70
iter 71
iter 72
iter 73
iter 74
iter 75
iter 76
iter 77
iter 78
iter 79
iter 80
iter 81
iter 82
iter 83
iter 84
iter 85
iter 86
iter 87
iter 88
iter 89
iter 90
iter 91
iter 92
iter 93
iter 94
iter 95
iter 96
iter 97
iter 98
iter 99
iter 100
iter 101
iter 102
iter 103
iter 104
iter 105
iter 106
iter 107
iter 108
iter 109
iter 110
iter 111
iter 112
iter 113
iter 114
iter 115
iter 116
iter 117
iter 118
iter 119
iter 120
iter 121
iter 122
ite

In [8]:
n_records = expert_data['observations'].shape[0]
n_records

400000

In [9]:
import numpy as np
np.save('expert_data', expert_data)

In [10]:
expert_data['actions'].shape

(400000, 1, 3)

In [11]:
tf.reset_default_graph()

In [12]:
features = expert_data['observations']
labels = expert_data['actions']
labels = labels.reshape(-1, 3)

In [52]:
import keras
from keras.models import Sequential
import keras.backend as K
K.clear_session()

model = Sequential()

In [53]:
from keras.layers import Dense, Activation

model.add(Dense(units=128, input_dim=11))
model.add(Activation('relu'))
model.add(Dense(units=32))
model.add(Activation('relu'))
model.add(Dense(units=3))

In [54]:
model.compile(loss='mean_squared_error', optimizer='adam', 
              metrics=['mean_squared_error'])

In [55]:
history = model.fit(features, labels, batch_size=512,epochs=10)

Epoch 1/10
400000/400000 [==============================] - 2s - loss: 0.0950 - mean_squared_error: 0.0950      

In [19]:
print (history)

In [8]:
import pickle
import tensorflow as tf
import numpy as np
import tf_util
import gym
import load_policy
from dotmap import DotMap

def evaluate(model, envname, render=True, max_timesteps=None, 
             obs_dim=8, action_dim=3, num_rollouts=20):
    """
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument('expert_policy_file', type=str)
    parser.add_argument('envname', type=str)
    parser.add_argument('--render', action='store_true')
    parser.add_argument("--max_timesteps", type=int)
    parser.add_argument('--num_rollouts', type=int, default=20,
                        help='Number of expert roll outs')
    args = parser.parse_args()
    """
    args = {'envname': envname, 'render': render, 
           'max_timesteps': max_timesteps, 
           'num_rollouts': num_rollouts}
    args = DotMap(args)
    print('start evaluate!')

    import gym
    env = gym.make(args.envname)
    max_steps = args.max_timesteps or env.spec.timestep_limit

    returns = []
    observations = []
    actions = []
    for i in range(args.num_rollouts):
        print('iter', i)
        obs = env.reset()
        done = False
        totalr = 0.
        steps = 0
        while not done:
            obs = obs.reshape((1,obs_dim))
            action = model.predict(obs)
            action = action.reshape((1,action_dim))
            observations.append(obs)
            actions.append(action)
            obs, r, done, _ = env.step(action)
            totalr += r
            steps += 1
            if args.render:
                env.render()
            # if steps % 100 == 0: print("%i/%i"%(steps, max_steps))
            if steps >= max_steps:
                break
        returns.append(totalr)

    print('returns', returns)
    print('mean return', np.mean(returns))
    print('std of return', np.std(returns))

    evaluate_data = {'observations': np.array(observations),
                     'actions': np.array(actions)}
    return evaluate_data

In [56]:
evaluate_data = evaluate(model, 'Hopper-v1')

[2017-03-21 16:23:17,479] Making new env: Hopper-v1


start evaluate!
iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9
iter 10
iter 11
iter 12
iter 13
iter 14
iter 15
iter 16
iter 17
iter 18
iter 19
returns [1471.7647626188775, 1416.143807520049, 1504.2078395982753, 1477.1157949284941, 1478.0539511952084, 1405.8969434041196, 1059.8404936421171, 1509.602319922207, 1506.1907310745946, 1566.7741453621102, 1503.9340786292112, 1778.5365031729075, 1472.4788360547309, 2019.5387367263093, 1471.8375009556473, 1560.541544451451, 1796.5083937081295, 1496.0222335895498, 1553.1609105645903, 1497.2834759056047]
mean return 1527.27165015
std of return 179.535962546


## Build a smaller network

In [45]:
import keras
from keras.models import Sequential
import keras.backend as K
K.clear_session()

model = Sequential()

In [46]:
from keras.layers import Dense, Activation

model.add(Dense(units=128, input_dim=11))
model.add(Activation('relu'))
model.add(Dense(units=3))

In [47]:
model.compile(loss='mean_squared_error', optimizer='adam', 
              metrics=['mean_squared_error'])

In [50]:
history = model.fit(features, labels, batch_size=512, epochs=40)

Epoch 1/40
400000/400000 [==============================] - 2s - loss: 3.6050e-04 - mean_squared_error: 3.6050e-04      

In [51]:
evaluate_data = evaluate(model, 'Hopper-v1')

[2017-03-21 16:20:17,590] Making new env: Hopper-v1


start evaluate!
iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9
iter 10
iter 11
iter 12
iter 13
iter 14
iter 15
iter 16
iter 17
iter 18
iter 19
returns [2255.5107560506535, 1736.8333811609559, 1645.3655135014603, 1650.7535619638893, 1735.680847424185, 2179.1814711461702, 1600.9186210134169, 2108.7210358529105, 1939.5182495036072, 2498.8295411199902, 2083.8307252790923, 3100.841299156385, 2824.7756640698776, 1755.6265948832231, 2085.2779153304587, 2117.2240777146249, 2819.199692927571, 2268.568461462185, 2751.8523586149622, 2461.8277664645479]
mean return 2181.01687673
std of return 433.251366137


## Result

Baseline  
expert performance   
mean return 3778  
std of return 3.577  

Network Archtects:  
inputs dim 11  
1 hidden layer of 128 units with relu activation  
1 ouput layer with 3 units and linear activation  

training batch size = 512

| Epoches | Mean return | std of return |
|---------|-------------|---------------|
| 10      | 607.7       | 434.1         |
| 20      | 1523        | 207.5         |
| 40      | 2033.2      | 360.6         |
| 80      | 2181        | 433.3         |

Network Architects:  
Inputs dim 11  
2 hidden layers of hidden units 128 and 32 with relu activation  
1 output layer of 3 units with linear activation  

train epoch = 10

| Batch size | Mean return | std of return |
| ---------- |
| 32         | 3777        | 4             |
| 512        | 1527        | 179.5         |

# Change to another task

Task Specification

| Name          | observation dim | action dim |
| --------------|
| Hopper-v1     | 11              | 3          |
| humanoid-v1   | 376             | 17         |
| Ant-v1        | 111             | 8          |

In [74]:
import os
task_name = 'Ant-v1'
expert_dir = os.path.join('experts', task_name+'.pkl')

In [75]:
expert_data_2 = get_expert_data(expert_dir, task_name, 
                             render=False, num_rollouts=400)

loading and building expert policy
obs (1, 111) (1, 111)
loaded and built
Instructions for updating:
Please use tf.global_variables instead.


[2017-03-21 16:51:51,472] From /home/luo/dlnd/homework/hw1/tf_util.py:91: all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Please use tf.global_variables instead.


Instructions for updating:
Use `tf.variables_initializer` instead.


[2017-03-21 16:51:51,474] From /home/luo/dlnd/homework/hw1/tf_util.py:92: initialize_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.variables_initializer` instead.
[2017-03-21 16:51:51,479] Making new env: Ant-v1


iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9
iter 10
iter 11
iter 12
iter 13
iter 14
iter 15
iter 16
iter 17
iter 18
iter 19
iter 20
iter 21
iter 22
iter 23
iter 24
iter 25
iter 26
iter 27
iter 28
iter 29
iter 30
iter 31
iter 32
iter 33
iter 34
iter 35
iter 36
iter 37
iter 38
iter 39
iter 40
iter 41
iter 42
iter 43
iter 44
iter 45
iter 46
iter 47
iter 48
iter 49
iter 50
iter 51
iter 52
iter 53
iter 54
iter 55
iter 56
iter 57
iter 58
iter 59
iter 60
iter 61
iter 62
iter 63
iter 64
iter 65
iter 66
iter 67
iter 68
iter 69
iter 70
iter 71
iter 72
iter 73
iter 74
iter 75
iter 76
iter 77
iter 78
iter 79
iter 80
iter 81
iter 82
iter 83
iter 84
iter 85
iter 86
iter 87
iter 88
iter 89
iter 90
iter 91
iter 92
iter 93
iter 94
iter 95
iter 96
iter 97
iter 98
iter 99
iter 100
iter 101
iter 102
iter 103
iter 104
iter 105
iter 106
iter 107
iter 108
iter 109
iter 110
iter 111
iter 112
iter 113
iter 114
iter 115
iter 116
iter 117
iter 118
iter 119
iter 120
iter 121
iter 122
ite

In [76]:
np.save(task_name+'_data', expert_data_2)

In [77]:
expert_data_2['observations'].shape

(398485, 111)

In [78]:
expert_data_2['actions'].shape

(398485, 1, 8)

In [86]:
features = expert_data_2['observations']
labels = expert_data_2['actions']
labels = labels.reshape((-1, 8))

In [82]:
import keras
from keras.models import Sequential
import keras.backend as K
K.clear_session()

model = Sequential()

In [83]:
from keras.layers import Dense, Activation

model.add(Dense(units=1024, input_dim=111))
model.add(Activation('relu'))
model.add(Dense(units=256))
model.add(Activation('relu'))
model.add(Dense(units=8))

In [84]:
model.compile(loss='mean_squared_error', optimizer='adam', 
              metrics=['mean_squared_error'])

In [87]:
history = model.fit(features, labels, batch_size=128,epochs=10)

Epoch 1/10
398485/398485 [==============================] - 11s - loss: 0.0028 - mean_squared_error: 0.0028      

In [88]:
print (history)

In [91]:
evaluate_data = evaluate(model, task_name, obs_dim=111, action_dim=8)

[2017-03-21 17:08:45,664] Making new env: Ant-v1


start evaluate!
iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9
iter 10
iter 11
iter 12
iter 13
iter 14
iter 15
iter 16
iter 17
iter 18
iter 19
returns [4634.0834827141252, 5039.7127255033647, 4840.2418694641037, 4822.1964591491114, 4887.3494374827033, 4714.770038694186, 4779.3704994362097, 4769.7250347553181, 4759.5163693122513, 4745.0480721102413, 4838.7320177555448, 4977.2912405240704, 4787.1439609318586, 4645.0442547319071, 4841.7853003817318, 5013.8090835935027, 4808.2627855371811, 4864.13934539191, 4788.6520307560477, 4937.7219807562233]
mean return 4824.72979945
std of return 105.738751386


Task: Ant-v1  
Expert performance:  
Mean Return = 4809  
std of return = 252.9  

Neural Net performance with hidden layer 1024 and 256 units  
Mean return = 4824    
std of return = 105.7     

In [9]:
import os
task_name = 'Humanoid-v1'
expert_dir = os.path.join('experts', task_name+'.pkl')

In [10]:
expert_data_3 = get_expert_data(expert_dir, task_name, 
                             render=False, num_rollouts=100)

loading and building expert policy
obs (1, 376) (1, 376)
loaded and built
Instructions for updating:
Please use tf.global_variables instead.


[2017-03-22 10:38:25,628] From /home/luo/dlnd/homework/hw1/tf_util.py:91: all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Please use tf.global_variables instead.


Instructions for updating:
Use `tf.variables_initializer` instead.


[2017-03-22 10:38:25,630] From /home/luo/dlnd/homework/hw1/tf_util.py:92: initialize_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.variables_initializer` instead.
[2017-03-22 10:38:25,644] Making new env: Humanoid-v1


iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9
iter 10
iter 11
iter 12
iter 13
iter 14
iter 15
iter 16
iter 17
iter 18
iter 19
iter 20
iter 21
iter 22
iter 23
iter 24
iter 25
iter 26
iter 27
iter 28
iter 29
iter 30
iter 31
iter 32
iter 33
iter 34
iter 35
iter 36
iter 37
iter 38
iter 39
iter 40
iter 41
iter 42
iter 43
iter 44
iter 45
iter 46
iter 47
iter 48
iter 49
iter 50
iter 51
iter 52
iter 53
iter 54
iter 55
iter 56
iter 57
iter 58
iter 59
iter 60
iter 61
iter 62
iter 63
iter 64
iter 65
iter 66
iter 67
iter 68
iter 69
iter 70
iter 71
iter 72
iter 73
iter 74
iter 75
iter 76
iter 77
iter 78
iter 79
iter 80
iter 81
iter 82
iter 83
iter 84
iter 85
iter 86
iter 87
iter 88
iter 89
iter 90
iter 91
iter 92
iter 93
iter 94
iter 95
iter 96
iter 97
iter 98
iter 99
returns [10419.923861695288, 10450.797896603946, 10445.420706178073, 10427.9814855075, 10223.61357941863, 10381.084975910815, 10397.529300193613, 10421.681565359326, 10427.294229212534, 10438.436568292949, 1044

In [98]:
features = expert_data_3['observations'].astype(np.float32)
labels = expert_data_3['actions'].astype(np.float32)
expert_data_3['observations'] = features
expert_data_3['actions'] = labels
labels = labels.reshape((-1, 17))

In [115]:
np.save(task_name+'_data', expert_data_3)

In [12]:
expert_data_3['observations'].shape

(100000, 376)

In [13]:
expert_data_3['actions'].shape

(100000, 1, 17)

In [14]:
obs_dim = 376
action_dim = 17

In [99]:
features = expert_data_3['observations']
labels = expert_data_3['actions']
labels = labels.reshape((-1, 17))

In [100]:
import keras
from keras.models import Sequential
import keras.backend as K
K.clear_session()

model = Sequential()

In [101]:
from keras.layers import Dense, Activation

model.add(Dense(units=2048, input_dim=376))
model.add(Activation('relu'))
model.add(Dense(units=256))
model.add(Activation('relu'))
model.add(Dense(units=17))

In [102]:
model.compile(loss='mean_squared_error', optimizer='adam', 
              metrics=['mean_squared_error'])

In [103]:
history = model.fit(features, labels, batch_size=128,epochs=20)

Epoch 1/20
100000/100000 [==============================] - 5s - loss: 6.5497 - mean_squared_error: 6.5497          

In [104]:
print (history)

In [146]:
evaluate_data = evaluate(model, task_name, num_rollouts=100, render=False,
                         obs_dim=obs_dim, action_dim=action_dim)

[2017-03-22 12:18:28,249] Making new env: Humanoid-v1


start evaluate!
iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9
iter 10
iter 11
iter 12
iter 13
iter 14
iter 15
iter 16
iter 17
iter 18
iter 19
iter 20
iter 21
iter 22
iter 23
iter 24
iter 25
iter 26
iter 27
iter 28
iter 29
iter 30
iter 31
iter 32
iter 33
iter 34
iter 35
iter 36
iter 37
iter 38
iter 39
iter 40
iter 41
iter 42
iter 43
iter 44
iter 45
iter 46
iter 47
iter 48
iter 49
iter 50
iter 51
iter 52
iter 53
iter 54
iter 55
iter 56
iter 57
iter 58
iter 59
iter 60
iter 61
iter 62
iter 63
iter 64
iter 65
iter 66
iter 67
iter 68
iter 69
iter 70
iter 71
iter 72
iter 73
iter 74
iter 75
iter 76
iter 77
iter 78
iter 79
iter 80
iter 81
iter 82
iter 83
iter 84
iter 85
iter 86
iter 87
iter 88
iter 89
iter 90
iter 91
iter 92
iter 93
iter 94
iter 95
iter 96
iter 97
iter 98
iter 99
returns [10392.002031821541, 10410.591247860541, 10366.2659866431, 10328.081439130638, 10213.784185652066, 10343.607543079053, 10249.05418851833, 10425.566747696375, 10347.665307891655, 10287.65

In [131]:
evaluate_data['observations'].shape

(89919, 1, 376)

In [144]:
model.save('Humanoid-model')

## Ask the expert to label the data

In [65]:
import load_policy
import tf_util

def label_data(data, task_name, obs_dim, action_dim):
    features = data['observations']
    policy_fn = load_policy.load_policy(os.path.join('experts', 
                                                     task_name+'.pkl'))
    batch_size = 128
    num = features.shape[0]
    labels = np.zeros_like(data['actions'])
    with tf.Session():
        tf_util.initialize()
        for i in range(0, num, batch_size):
            labels[i:i+batch_size] = \
                policy_fn(features[i:i+batch_size].\
                          reshape((-1,obs_dim))).\
                          reshape((-1, 1, action_dim))
            
    return {'observations': features, 'actions': labels}

In [132]:
dagger_data = label_data(evaluate_data, task_name, obs_dim, 17)

obs (1, 376) (1, 376)
Instructions for updating:
Please use tf.global_variables instead.


[2017-03-22 12:02:09,227] From /home/luo/dlnd/homework/hw1/tf_util.py:91: all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Please use tf.global_variables instead.


Instructions for updating:
Use `tf.variables_initializer` instead.


[2017-03-22 12:02:09,228] From /home/luo/dlnd/homework/hw1/tf_util.py:92: initialize_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.variables_initializer` instead.


## Train the model again


In [133]:
print (features.shape)
print (labels.shape)

(149762, 376)
(149762, 17)


In [135]:
# Combined last time feature and add into daggar

features_daggar = dagger_data['observations'].reshape((-1, obs_dim))
labels_daggar = dagger_data['actions'].reshape((-1,action_dim))

features = np.vstack((features_daggar,features))
labels = np.vstack((labels_daggar,labels))

In [136]:
print (features_daggar.shape)
print (labels_daggar.shape)

(89919, 376)
(89919, 17)


In [137]:
print (features.shape)
print (labels.shape)

(239681, 376)
(239681, 17)


In [138]:
import keras
from keras.models import Sequential
import keras.backend as K
K.clear_session()

model = Sequential()

In [139]:
from keras.layers import Dense, Activation

model.add(Dense(units=2048, input_dim=376))
model.add(Activation('relu'))
model.add(Dense(units=256))
model.add(Activation('relu'))
model.add(Dense(units=17))

In [140]:
model.compile(loss='mean_squared_error', optimizer='adam', 
              metrics=['mean_squared_error'])

In [141]:
print (features.shape)
print (labels.shape)

(239681, 376)
(239681, 17)


In [145]:
history = model.fit(features, labels, 
                    batch_size=128,epochs=20)

Epoch 1/20
239681/239681 [==============================] - 13s - loss: 0.0301 - mean_squared_error: 0.0301     

## Result on task Humanoid-v1

Without Daggar    

Expert performance  
Mean return = 10402  
std of return = 53  

network with hidden layer [2048, 256]  
train on expert rollouts of 400  
with 400000 entries data  

|  Epochs  |  mean return  |  std of return  |
| -------- |
|  10      |  4715         |  3732           |
|  20      |  8136         |  2530           |
|  40      |  9434         |  2778           |


With Daggar

Same network with hidden layer [2048, 256]  
first train on expert rollout of 100  
with 100000 entries data  

|Daggar stage|Rollouts add|Data add|Total Data|Epoch|Mean return|std  |
|------------|
|No Dagger   |100         |100000  |100000    |20   |744.7      |221.9|
|1           |100         |10818   |110818    |20   |3570       |2347 |
|2           |100         |38944   |149762    |20   |9157       |2537 |
|3           |100         |89919   |239681    |20   |10258      |1098 |
|            |            |        |          |40   |10279      |622.6|